In [1]:
from datasets import Dataset, Audio
import pandas as pd
from transformers import WhisperProcessor
from transformers import WhisperForConditionalGeneration, TrainingArguments, Trainer

/home/sam/Documents/tamil_whisper_stt/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("dataset/metadata.csv")
df["file"] = df["file"].apply(lambda x: f"dataset/wavs_converted/{x}.wav")
dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("file", Audio(sampling_rate=16000))

In [3]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="ta", task="transcribe")

def prepare(example):
    audio = example["file"]
    inputs = processor(audio["array"], sampling_rate=16000, return_tensors="pt")
    input_values = inputs.input_features.squeeze(0)
    labels = processor.tokenizer(example["text"], return_tensors="pt").input_ids.squeeze(0)
    return {"input_features": input_values, "labels": labels}

processed_dataset = dataset.map(prepare)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5395/5395 [06:07<00:00, 14.69 examples/s]


In [4]:
from dataclasses import dataclass
import torch

@dataclass
class CustomDataCollator:
    processor: any

    def __call__(self, features):
        # Extract input features and labels
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        # Pad input features
        batch = self.processor.feature_extractor.pad(
            input_features, return_tensors="pt"
        )

        # Pad labels directly using tokenizer
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            return_tensors="pt",
            padding=True
        )

        # Replace padding token ids with -100 for loss masking
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100
        )

        batch["labels"] = labels
        return batch

In [5]:
data_collator = CustomDataCollator(processor=processor)

In [6]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

training_args = TrainingArguments(
    output_dir="./whisper-tamil-finetuned",
    per_device_train_batch_size=8,
    learning_rate=1e-5,
    num_train_epochs=10,
    save_steps=500,
    fp16=True,
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    tokenizer=processor.feature_extractor,
    data_collator = CustomDataCollator(processor=processor)
)

trainer.train()

/tmp/ipykernel_100321/1066871092.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,2.371100
20,1.137900
30,0.897300
40,0.767000
50,0.763300
60,0.668800
70,0.617300
80,0.617000
90,0.574500
100,0.534800


/home/sam/Documents/tamil_whisper_stt/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=6750, training_loss=0.094480023947027, metrics={'train_runtime': 2816.2182, 'train_samples_per_second': 19.157, 'train_steps_per_second': 2.397, 'total_flos': 1.328188852224e+18, 'train_loss': 0.094480023947027, 'epoch': 10.0})